In [4]:
import random
import numpy as np
from data_process import get_FASHION_data, get_MUSHROOM_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython|
%load_ext autoreload
%autoreload 2

# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [5]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [6]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

In [7]:
n_class_fashion

10

# Loading Mushroom

In the following cells we determine the splitting of the mushroom dataset.
<br /> TRAINING + VALIDATION = 0.8, TESTING = 0.2

In [8]:
# TRAINING = 0.6 indicates 60% of the data is used as the training dataset.
VALIDATION = 0.2

In [9]:
data = get_MUSHROOM_data(VALIDATION)
X_train_MR, y_train_MR = data['X_train'], data['y_train']
X_val_MR, y_val_MR = data['X_val'], data['y_val']
X_test_MR, y_test_MR = data['X_test'], data['y_test']
n_class_MR = len(np.unique(y_test_MR))

print("Number of train samples: ", X_train_MR.shape[0])
print("Number of val samples: ", X_val_MR.shape[0])
print("Number of test samples: ", X_test_MR.shape[0])

Number of train samples:  4874
Number of val samples:  1625
Number of test samples:  1625


### Get Accuracy

In [10]:
n_class_MR
X_train_fashion.shape

(50000, 784)

This function computes how well your model performs using accuracy as a metric.

In [11]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on Fashion-MNIST

In [151]:
lr = 0.75
n_epochs = 100
dim_fashion = X_train_fashion.shape[1]

percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

Training Perceptron...
Epoch 1/100, Accuracy: 49.73%
Epoch 2/100, Accuracy: 66.99%
Epoch 3/100, Accuracy: 69.22%
Epoch 4/100, Accuracy: 71.15%
Epoch 5/100, Accuracy: 72.70%
Epoch 6/100, Accuracy: 74.42%
Epoch 7/100, Accuracy: 75.26%
Epoch 8/100, Accuracy: 76.40%
Epoch 9/100, Accuracy: 76.55%
Epoch 10/100, Accuracy: 77.39%
Epoch 11/100, Accuracy: 76.62%
Epoch 12/100, Accuracy: 76.98%
Epoch 13/100, Accuracy: 76.21%
Epoch 14/100, Accuracy: 76.44%
Epoch 15/100, Accuracy: 77.13%
Epoch 16/100, Accuracy: 77.14%
Epoch 17/100, Accuracy: 77.81%
Epoch 18/100, Accuracy: 78.18%
Epoch 19/100, Accuracy: 78.66%
Epoch 20/100, Accuracy: 79.32%
Epoch 21/100, Accuracy: 79.46%
Epoch 22/100, Accuracy: 80.24%
Epoch 23/100, Accuracy: 80.22%
Epoch 24/100, Accuracy: 80.80%
Epoch 25/100, Accuracy: 80.66%
Epoch 26/100, Accuracy: 81.16%
Epoch 27/100, Accuracy: 81.03%
Epoch 28/100, Accuracy: 81.36%
Epoch 29/100, Accuracy: 81.27%
Epoch 30/100, Accuracy: 81.45%
Epoch 31/100, Accuracy: 81.36%
Epoch 32/100, Accuracy: 8

In [152]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 82.442000


### Validate Perceptron on Fashion-MNIST

In [153]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 81.440000


### Test Perceptron on Fashion-MNIST

In [154]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 81.110000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [155]:
output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

## Train Perceptron on Mushroom

In [147]:
lr = 0.25
n_epochs = 100

percept_MR = Perceptron(n_class_MR, lr, n_epochs)
percept_MR.train(X_train_MR, y_train_MR)

Training Perceptron...
Epoch 1/100, Accuracy: 51.68%
Epoch 2/100, Accuracy: 75.13%
Epoch 3/100, Accuracy: 48.32%
Epoch 4/100, Accuracy: 51.68%
Epoch 5/100, Accuracy: 65.10%
Epoch 6/100, Accuracy: 51.68%
Epoch 7/100, Accuracy: 48.28%
Epoch 8/100, Accuracy: 51.68%
Epoch 9/100, Accuracy: 64.20%
Epoch 10/100, Accuracy: 73.35%
Epoch 11/100, Accuracy: 74.89%
Epoch 12/100, Accuracy: 81.04%
Epoch 13/100, Accuracy: 72.73%
Epoch 14/100, Accuracy: 81.35%
Epoch 15/100, Accuracy: 73.23%
Epoch 16/100, Accuracy: 81.66%
Epoch 17/100, Accuracy: 75.42%
Epoch 18/100, Accuracy: 82.66%
Epoch 19/100, Accuracy: 74.48%
Epoch 20/100, Accuracy: 82.70%
Epoch 21/100, Accuracy: 77.35%
Epoch 22/100, Accuracy: 84.63%
Epoch 23/100, Accuracy: 76.26%
Epoch 24/100, Accuracy: 84.04%
Epoch 25/100, Accuracy: 76.84%
Epoch 26/100, Accuracy: 84.65%
Epoch 27/100, Accuracy: 77.21%
Epoch 28/100, Accuracy: 84.82%
Epoch 29/100, Accuracy: 77.19%
Epoch 30/100, Accuracy: 84.86%
Epoch 31/100, Accuracy: 77.33%
Epoch 32/100, Accuracy: 8

In [148]:
pred_percept = percept_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_MR)))

The training accuracy is given by: 87.833402


### Validate Perceptron on Mushroom

In [149]:
pred_percept = percept_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_MR)))

The validation accuracy is given by: 87.569231


### Test Perceptron on Mushroom

In [150]:
pred_percept = percept_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_MR)))

The testing accuracy is given by: 85.538462


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on Fashion-MNIST

In [139]:
lr = 0.15
n_epochs = 50
reg_const = 0.01
batch_size = 1024

svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const, batch_size)
svm_fashion.train(X_train_fashion, y_train_fashion)
svm_fashion.load_weights()

Training SVM...
Epoch 1/50, Accuracy: 75.31
Epoch 2/50, Accuracy: 79.51
Epoch 3/50, Accuracy: 82.62
Epoch 4/50, Accuracy: 77.40
Epoch 5/50, Accuracy: 79.30
Epoch 6/50, Accuracy: 82.53
Epoch 7/50, Accuracy: 82.36
Epoch 8/50, Accuracy: 82.23
Epoch 9/50, Accuracy: 83.14
Epoch 10/50, Accuracy: 81.86
Epoch 11/50, Accuracy: 77.39
Epoch 12/50, Accuracy: 81.13
Epoch 13/50, Accuracy: 79.76
Epoch 14/50, Accuracy: 81.57
Epoch 15/50, Accuracy: 83.07
Epoch 16/50, Accuracy: 83.16
Epoch 17/50, Accuracy: 83.00
Epoch 18/50, Accuracy: 84.20
Epoch 19/50, Accuracy: 83.46
Epoch 20/50, Accuracy: 79.71
Epoch 21/50, Accuracy: 78.17
Epoch 22/50, Accuracy: 84.06
Epoch 23/50, Accuracy: 80.84
Epoch 24/50, Accuracy: 84.82
Epoch 25/50, Accuracy: 81.19
Epoch 26/50, Accuracy: 82.95
Epoch 27/50, Accuracy: 79.40
Epoch 28/50, Accuracy: 78.94
Epoch 29/50, Accuracy: 81.33
Epoch 30/50, Accuracy: 79.37
Epoch 31/50, Accuracy: 82.37
Epoch 32/50, Accuracy: 81.50
Epoch 33/50, Accuracy: 80.01
Epoch 34/50, Accuracy: 79.22
Epoch 3

In [140]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 85.060000


### Validate SVM on Fashion-MNIST

In [141]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 84.190000


### Test SVM on Fashion-MNIST

In [142]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 83.160000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [143]:
output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Mushroom

In [31]:
lr = 0.5
n_epochs = 10
reg_const = 0.001

svm_MR = SVM(n_class_MR, lr, n_epochs, reg_const)
svm_MR.train(X_train_MR, y_train_MR)

Training SVM...
Epoch 1/10, Accuracy: 86.42%
Epoch 2/10, Accuracy: 86.99%
Epoch 3/10, Accuracy: 81.86%
Epoch 4/10, Accuracy: 88.70%
Epoch 5/10, Accuracy: 89.70%
Epoch 6/10, Accuracy: 89.43%
Epoch 7/10, Accuracy: 90.44%
Epoch 8/10, Accuracy: 79.79%
Epoch 9/10, Accuracy: 85.86%
Epoch 10/10, Accuracy: 89.95%


In [16]:
pred_svm = svm_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_MR)))

The training accuracy is given by: 89.946656


### Validate SVM on Mushroom

In [195]:
pred_svm = svm_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_MR)))

The validation accuracy is given by: 88.369231


## Test SVM on Mushroom

In [196]:
pred_svm = svm_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_MR)))

The testing accuracy is given by: 88.184615


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on Fashion-MNIST

In [ ]:
lr = 0.5
n_epochs = 10
reg_const = 0.5

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

In [ ]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

### Validate Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

### Testing Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Mushroom

In [ ]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

softmax_MR = Softmax(n_class_MR, lr, n_epochs, reg_const)
softmax_MR.train(X_train_MR, y_train_MR)

In [ ]:
pred_softmax = softmax_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_MR)))

### Validate Softmax on Mushroom

In [ ]:
pred_softmax = softmax_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_MR)))

### Testing Softmax on Mushroom

In [ ]:
pred_softmax = softmax_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_MR)))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Training Logistic Classifer

In [32]:
y_train_MR

array([1, 0, 0, ..., 1, 1, 0], dtype=int64)

In [64]:
learning_rate = 0.7
n_epochs = 100
threshold = 0.5

lr = Logistic(learning_rate, n_epochs, threshold)
lr.train(X_train_MR, y_train_MR)

0.9969806630845547 0.0009926191695871773
Epoch 1/100, Accuracy: 87.34%
1.0 0.0
Epoch 2/100, Accuracy: 88.72%
1.0 0.0
Epoch 3/100, Accuracy: 89.52%
1.0 0.0
Epoch 4/100, Accuracy: 91.24%
1.0 0.0
Epoch 5/100, Accuracy: 92.57%
1.0 0.0
Epoch 6/100, Accuracy: 92.43%
1.0 0.0
Epoch 7/100, Accuracy: 92.57%
1.0 0.0
Epoch 8/100, Accuracy: 92.39%
1.0 0.0
Epoch 9/100, Accuracy: 92.78%
1.0 0.0
Epoch 10/100, Accuracy: 92.41%
1.0 0.0
Epoch 11/100, Accuracy: 92.94%
1.0 0.0
Epoch 12/100, Accuracy: 92.72%
1.0 0.0
Epoch 13/100, Accuracy: 93.58%
1.0 0.0
Epoch 14/100, Accuracy: 90.93%
1.0 0.0
Epoch 15/100, Accuracy: 91.79%
1.0 0.0
Epoch 16/100, Accuracy: 94.19%
1.0 0.0
Epoch 17/100, Accuracy: 93.19%
1.0 0.0
Epoch 18/100, Accuracy: 91.34%
1.0 0.0
Epoch 19/100, Accuracy: 91.57%
1.0 0.0
Epoch 20/100, Accuracy: 90.19%
1.0 0.0
Epoch 21/100, Accuracy: 95.45%
1.0 0.0
Epoch 22/100, Accuracy: 93.64%
1.0 0.0
Epoch 23/100, Accuracy: 95.42%
1.0 0.0
Epoch 24/100, Accuracy: 90.77%
1.0 0.0
Epoch 25/100, Accuracy: 86.56%
1

c:\Users\touba\OneDrive\桌面\CS444-MP\assignment1\models\logistic.py:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Epoch 64/100, Accuracy: 91.20%
1.0 0.0
Epoch 65/100, Accuracy: 93.39%
1.0 0.0
Epoch 66/100, Accuracy: 94.77%
1.0 0.0
Epoch 67/100, Accuracy: 94.77%
1.0 0.0
Epoch 68/100, Accuracy: 94.05%
1.0 0.0
Epoch 69/100, Accuracy: 92.53%
1.0 0.0
Epoch 70/100, Accuracy: 92.59%
1.0 0.0
Epoch 71/100, Accuracy: 94.60%
1.0 0.0
Epoch 72/100, Accuracy: 90.95%
1.0 0.0
Epoch 73/100, Accuracy: 92.76%
1.0 0.0
Epoch 74/100, Accuracy: 94.65%
1.0 0.0
Epoch 75/100, Accuracy: 93.62%
1.0 0.0
Epoch 76/100, Accuracy: 91.22%
1.0 0.0
Epoch 77/100, Accuracy: 92.76%
1.0 0.0
Epoch 78/100, Accuracy: 91.83%
1.0 0.0
Epoch 79/100, Accuracy: 93.29%
1.0 0.0
Epoch 80/100, Accuracy: 94.11%
1.0 0.0
Epoch 81/100, Accuracy: 94.21%
1.0 0.0
Epoch 82/100, Accuracy: 94.52%
1.0 0.0
Epoch 83/100, Accuracy: 92.49%
1.0 0.0
Epoch 84/100, Accuracy: 89.45%
1.0 0.0
Epoch 85/100, Accuracy: 91.57%
1.0 0.0
Epoch 86/100, Accuracy: 95.06%
1.0 0.0
Epoch 87/100, Accuracy: 95.18%
1.0 0.0
Epoch 88/100, Accuracy: 94.01%
1.0 0.0
Epoch 89/100, Accuracy: 9

In [65]:
pred_lr = lr.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_MR)))

The training accuracy is given by: 94.460402


c:\Users\touba\OneDrive\桌面\CS444-MP\assignment1\models\logistic.py:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


### Validate Logistic Classifer

In [66]:
pred_lr = lr.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_MR)))

The validation accuracy is given by: 94.215385


### Test Logistic Classifier

In [67]:
pred_lr = lr.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_MR)))

The testing accuracy is given by: 94.584615
